In [7]:
import pandas as pd 
import matplotlib.pyplot as plt
import nltk

In [2]:
cd ..

/mnt/e/rt_emotional_recognition


In [3]:
df = pd.read_csv('artifacts/Data/ISEAR_dataset.csv')

In [4]:
df.head(20)

,sentiment,content,Unnamed: 2
0,joy,On days when I feel close to my partner and ot...,NaN
1,fear,Every time I imagine that someone I love or I ...,NaN
2,anger,When I had been obviously unjustly treated and...,NaN
3,sadness,When I think about the short time that we live...,NaN
4,disgust,At a gathering I found myself involuntarily si...,NaN
5,shame,When I realized that I was directing the feeli...,NaN
6,guilt,I feel guilty when when I realize that I consi...,NaN
7,joy,After my girlfriend had taken her exam we went...,NaN
8,fear,"When, for the first time I realized the meanin...",NaN
9,anger,When a car is overtaking another and I am forc...,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516 entries, 0 to 7515
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentiment   7516 non-null   object
 1   content     7516 non-null   object
 2   Unnamed: 2  3 non-null      object
dtypes: object(3)
memory usage: 176.3+ KB


In [6]:
df['sentiment'].unique()

array(['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt',
       'guit'], dtype=object)

In [8]:
df.columns = ['EMOTION', 'TEXT', 'Unnamed']


In [9]:
df = df.drop(columns=['Unnamed'])

In [10]:
df.head()

,EMOTION,TEXT
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [19]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [13]:
df['TEXT'] = df['TEXT'].apply(lambda a: " ".join(a.lower() for a in a.split()))

In [14]:
df['TEXT'] = df['TEXT'].apply(lambda a: " ".join(a.replace('[^\w\s\]', "") for a in a.split()))

In [17]:
stop = stopwords.words('english')
df['TEXT'] = df['TEXT'].apply(lambda a: " ".join(a for a in a.split() if a not in stop))

In [16]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [20]:
st = PorterStemmer()

In [23]:
df['TEXT'] = df['TEXT'].apply(lambda a: " ".join([st.stem(word) for word in a.split()]))

In [27]:
from spellchecker import SpellChecker
spell = SpellChecker()
df['TEXT'] = df['TEXT'].apply(lambda a: " ".join(spell.correction(word) if spell.correction(word) is not None else word for word in a.split()))

In [28]:
df.head()

,EMOTION,TEXT
0,joy,day feel close partner friends feel peace also...
1,fear,ever time imagine someone love could contact s...
2,anger,obvious unjustly treat possible lucid this
3,sadness,think short time live relax period life think ...
4,disgust,gather found involuntarily sit next two peopl ...


In [29]:
df['EMOTION'].value_counts()

EMOTION
joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1049
guit          1
Name: count, dtype: int64

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
le = LabelEncoder()

In [34]:
df['EMOTION'] = le.fit_transform(df['EMOTION'])

In [35]:
df['EMOTION'].value_counts()

EMOTION
5    1092
6    1082
0    1079
2    1076
7    1071
1    1066
3    1049
4       1
Name: count, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split

# Get the emotions with at least two instances
valid_emotions = df['EMOTION'].value_counts()[df['EMOTION'].value_counts() > 1].index.tolist()

# Filter the data to only include valid emotions
filtered_data = df[df['EMOTION'].isin(valid_emotions)]

# Split the filtered data into train and test sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(filtered_data['TEXT'], filtered_data['EMOTION'], stratify=filtered_data['EMOTION'])

In [38]:
from sklearn.feature_extraction. text import CountVectorizer, TfidfVectorizer

In [39]:
# counting Vectors 
cv = CountVectorizer()

cv.fit(df['TEXT'])
cv_xtrain = cv.transform(Xtrain)
cv_xtest = cv.transform(Xtest)

In [40]:
# tfidvec

tv = TfidfVectorizer()
tv.fit(df['TEXT'])
tv_xtrain = tv.transform(Xtrain)
tv_xtest = tv.transform(Xtest)

In [41]:
def build_model(model_initializer, independent_variables, target, independent_variables_test):
    model_initializer.fit(independent_variables, target)
    modelPred = model_initializer.predict(independent_variables_test)
    return metrics.accuracy_score(modelPred, Ytest)

In [44]:
from sklearn import model_selection, linear_model, naive_bayes, svm, metrics

In [47]:
output = build_model(naive_bayes.MultinomialNB(), cv_xtrain, Ytrain, cv_xtest)
print(output)

0.5689196381053752


In [48]:
output = build_model(naive_bayes.MultinomialNB(), tv_xtrain, Ytrain, tv_xtest)
print(output)

0.5678552421500799


In [49]:
from sklearn.linear_model import LogisticRegression

# Set a higher max_iter value
max_iter_value = 1000  # You can adjust this value based on your data

# for CV
output_cv = build_model(LogisticRegression(max_iter=max_iter_value), cv_xtrain, Ytrain, cv_xtest) 
print(output_cv)

# for TF-IDF
output_tfidf = build_model(LogisticRegression(max_iter=max_iter_value), tv_xtrain, Ytrain, tv_xtest)
print(output_tfidf)

0.5721128259712613
0.5864821713677488


In [51]:

output =build_model(svm. SVC(), cv_xtrain, Ytrain, cv_xtest)
print (output)

#for TF-IDF

output =build_model(svm.SVC(), tv_xtrain, Ytrain, tv_xtest)
print(output)

0.5481639169771155
0.5790313996806812


In [55]:
output = build_model(ensemble.RandomForestClassifier(), cv_xtrain, Ytrain, cv_xtest) 
print(output)

#for TF-IDF

output = build_model(ensemble.RandomForestClassifier(), tv_xtrain, Ytrain,tv_xtest)

print(output)

0.5428419372006387
0.5598722724853645


In [54]:
from sklearn import ensemble